# Deer Detection Model Training - Google Colab

Train a YOLOv8 deer detection model using Google Colab's free GPU.

**Before running:**
1. Go to `Runtime` → `Change runtime type`
2. Select `T4 GPU` as Hardware accelerator
3. Click `Save`

## 1. Setup Environment

In [ ]:
# Install required packages
!pip install -q ultralytics

In [ ]:
# Import libraries
from google.colab import drive
import os
from pathlib import Path
import shutil
import yaml
from ultralytics import YOLO

print("✓ Imports successful")

## 2. Mount Google Drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set paths
drive_root = Path('/content/drive/MyDrive')
project_folder = drive_root / 'Deer video detection'

print(f"✓ Google Drive mounted")
print(f"Project folder: {project_folder}")

## 3. Prepare Dataset

In [ ]:
# Set up paths for training data
# The train-model endpoint exports data to the root of the Drive folder
annotations_file = project_folder / 'annotations.json'
images_dir = project_folder / 'images'

# Verify data exists
if not annotations_file.exists():
    print("❌ annotations.json not found!")
    print(f"   Expected: {annotations_file}")
    print("\n📝 Run the 'Train Model' button in the web app first!")
    raise FileNotFoundError("Training data not found")

if not images_dir.exists():
    print("❌ images folder not found!")
    print(f"   Expected: {images_dir}")
    raise FileNotFoundError("Images folder not found")

image_count = len(list(images_dir.glob('*.jpg')))
print(f"✓ Found {image_count} training images")
print(f"✓ Annotations file: {annotations_file}")

In [ ]:
# Convert COCO to YOLO format
import json
from PIL import Image

# Load COCO annotations
print("Loading COCO annotations...")
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

print(f"  Images: {len(coco_data['images'])}")
print(f"  Annotations: {len(coco_data['annotations'])}")
print(f"  Categories: {len(coco_data['categories'])}")

# Create working directory and output directories
work_dir = Path('/content/deer-detection')
dataset_dir = work_dir / 'dataset'
train_images_dir = dataset_dir / 'images' / 'train'
train_labels_dir = dataset_dir / 'labels' / 'train'
val_images_dir = dataset_dir / 'images' / 'val'
val_labels_dir = dataset_dir / 'labels' / 'val'

for d in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Image ID to info mapping
image_id_to_info = {img['id']: img for img in coco_data['images']}

# Image ID to annotations mapping
image_id_to_annots = {}
for annot in coco_data['annotations']:
    image_id = annot['image_id']
    if image_id not in image_id_to_annots:
        image_id_to_annots[image_id] = []
    image_id_to_annots[image_id].append(annot)

# Split into train/val (80/20)
import random
all_image_ids = list(image_id_to_info.keys())
random.shuffle(all_image_ids)
split_idx = int(len(all_image_ids) * 0.8)
train_ids = set(all_image_ids[:split_idx])
val_ids = set(all_image_ids[split_idx:])

print(f"\nDataset split:")
print(f"  Train: {len(train_ids)} images")
print(f"  Val: {len(val_ids)} images")

# Convert each image
processed = 0
skipped = 0

for image_id, image_info in image_id_to_info.items():
    # Get image file
    image_filename = Path(image_info['file_name']).name
    image_path = images_dir / image_filename
    
    if not image_path.exists():
        skipped += 1
        continue
    
    # Get annotations
    annotations = image_id_to_annots.get(image_id, [])
    if not annotations:
        skipped += 1
        continue
    
    # Determine if train or val
    if image_id in train_ids:
        dest_images_dir = train_images_dir
        dest_labels_dir = train_labels_dir
    else:
        dest_images_dir = val_images_dir
        dest_labels_dir = val_labels_dir
    
    # Get image dimensions
    width = image_info['width']
    height = image_info['height']
    
    # Convert to YOLO format
    yolo_annotations = []
    for annot in annotations:
        bbox = annot['bbox']  # [x, y, width, height]
        x, y, w, h = bbox
        
        # Convert to YOLO format [class, x_center, y_center, width, height] normalized
        x_center = (x + w / 2) / width
        y_center = (y + h / 2) / height
        norm_width = w / width
        norm_height = h / height
        
        yolo_annotations.append(f"0 {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")
    
    # Copy image
    shutil.copy2(image_path, dest_images_dir / image_filename)
    
    # Save label
    label_filename = Path(image_filename).stem + ".txt"
    with open(dest_labels_dir / label_filename, 'w') as f:
        f.write('\n'.join(yolo_annotations))
    
    processed += 1

print(f"\n✓ Conversion complete!")
print(f"  Processed: {processed} images")
print(f"  Skipped: {skipped} images (no annotations)")

In [ ]:
# Create dataset.yaml
dataset_yaml = dataset_dir / 'dataset.yaml'

# Use absolute paths for reliability
yaml_content = {
    'path': str(dataset_dir.absolute()),
    'train': 'images/train',
    'val': 'images/val',
    'names': {0: 'deer'},
    'nc': 1
}

with open(dataset_yaml, 'w') as f:
    yaml.dump(yaml_content, f, default_flow_style=False)

print("✓ Created dataset.yaml")
print(f"  Path: {dataset_yaml}")
print(f"\nDataset configuration:")
for key, value in yaml_content.items():
    print(f"  {key}: {value}")

## 4. Train Model

In [ ]:
# Verify GPU is available
import torch

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: No GPU detected. Training will be slow!")
    print("Go to Runtime → Change runtime type → Select T4 GPU")

In [ ]:
# Initialize model
model = YOLO('yolov8n.pt')  # YOLOv8 nano (smallest/fastest)

print("✓ Model initialized")

In [ ]:
# Train the model with better error handling
try:
    # Verify dataset exists
    print(f"Dataset config: {dataset_yaml}")
    print(f"Dataset exists: {dataset_yaml.exists()}")
    
    # Check GPU
    device = 0 if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    
    # Start training
    results = model.train(
        data=str(dataset_yaml),
        epochs=100,
        imgsz=640,
        batch=8,  # Reduced from 16 for stability
        patience=50,  # Early stopping
        save_period=10,  # Save checkpoint every 10 epochs
        device=device,
        project=str(work_dir / 'runs'),
        name='deer_detector',
        exist_ok=True,
        verbose=True,
        workers=2  # Reduced for Colab stability
    )
    print("✓ Training completed successfully!")
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()
    
    # Debug info
    print("\nDebug info:")
    print(f"Dataset dir exists: {dataset_dir.exists()}")
    for split in ['train', 'val', 'test']:
        split_dir = dataset_dir / 'images' / split
        print(f"{split} exists: {split_dir.exists()}, images: {len(list(split_dir.glob('*.jpg'))) if split_dir.exists() else 0}")

## 5. Evaluate Model

In [ ]:
# Evaluate on test set
metrics = model.val(data=str(dataset_yaml), split='test')

print(f"\n{'='*50}")
print("Test Set Results:")
print(f"{'='*50}")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.p[0]:.3f}")
print(f"Recall: {metrics.box.r[0]:.3f}")

## 6. Test on Sample Images

In [ ]:
# Run inference on test images
from IPython.display import Image as IPImage, display

test_images_dir = dataset_dir / 'images/test'
test_images = list(test_images_dir.glob('*.jpg'))[:5]  # First 5 test images

for img_path in test_images:
    results = model(str(img_path))
    
    # Save annotated image
    annotated = results[0].plot()
    output_path = work_dir / f"test_{img_path.name}"
    import cv2
    cv2.imwrite(str(output_path), annotated)
    
    # Display
    print(f"\n{img_path.name}:")
    display(IPImage(filename=str(output_path), width=600))
    
    # Print detections
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            conf = box.conf[0]
            print(f"  Deer detected - Confidence: {conf:.2f}")
    else:
        print("  No deer detected")

## 7. Save Model to Google Drive

In [ ]:
# Copy best model to Google Drive
best_model = work_dir / 'runs/deer_detector/weights/best.pt'
output_model_dir = project_folder / 'trained_models'
output_model_dir.mkdir(exist_ok=True)

output_model_path = output_model_dir / 'deer_detector_best.pt'
shutil.copy2(best_model, output_model_path)

print(f"✓ Model saved to Google Drive: {output_model_path}")

# Also save last checkpoint
last_model = work_dir / 'runs/deer_detector/weights/last.pt'
if last_model.exists():
    shutil.copy2(last_model, output_model_dir / 'deer_detector_last.pt')
    print(f"✓ Last checkpoint saved")

# Copy training results
results_dir = output_model_dir / 'training_results'
results_dir.mkdir(exist_ok=True)

for file in ['results.png', 'confusion_matrix.png', 'results.csv']:
    src = work_dir / 'runs/deer_detector' / file
    if src.exists():
        shutil.copy2(src, results_dir / file)

print(f"✓ Training results saved to: {results_dir}")

## 8. Download Model to Local Computer

In [ ]:
# Download the model file
from google.colab import files

print("Downloading model file...")
files.download(str(best_model))
print("✓ Download complete!")
print("\nSave this file as: models/production/best.pt in your local project")

## 🎉 Training Complete!

### Next Steps:

1. **Download the model** (downloaded above) or access it from Google Drive:
   - `Deer video detection/trained_models/deer_detector_best.pt`

2. **Copy to your local project**:
   ```
   models/production/best.pt
   ```

3. **Test locally**:
   ```bash
   python src/inference/detector.py
   ```

4. **Configure Ring & Rainbird** and run the full system:
   ```bash
   python src/main.py
   ```